# Import Libraries & Files

In [ ]:
import pandas as pd
df_raw = pd.read_excel('/workspaces/cleandata/import_files/raw_data.xlsx')
df_code = pd.read_excel('/workspaces/cleandata/import_files/postcode_clean.xlsx', sheet_name='รหัสไปรษณีย์')

# Fix Wrong Postcode on Raw Data

## Total Raw Data

In [48]:
df_raw.shape[0]

1095

## Fix Raw Data That Not Included in Postcode Data

In [49]:
print("Postcode: " + str(set(df_raw['postcode']).difference(set(df_code['PostCode']))))
print("Province: " + str(set(df_raw['province_name']).difference(set(df_code['ProvinceThai']))))
print("District: " + str(set(df_raw['district_name']).difference(set(df_code['DistrictThaiShort']))))
print("Sub District: " + str(set(df_raw['sub_district_name']).difference(set(df_code['TambonThaiShort']))))

Postcode: set()
Province: set()
District: set()
Sub District: {'หนอง ปรือ'}


In [50]:
df_raw['sub_district_name'] = df_raw['sub_district_name'].str.replace('หนอง ปรือ','หนองปรือ')

## Create Column 'number' to Use as Key Join

In [51]:
df_raw['number'] = range(len(df_raw))

## Merge Data Between Raw Data and Postcode Data

In [86]:
df_merge = df_raw.merge(
    df_code[['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai']],
    left_on=['postcode', 'sub_district_name', 'district_name', 'province_name'],
    right_on=['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai'],
    how='left'
)

## List Data Which Is Not Match With Postcode Data

In [87]:
list_not_match = df_merge.loc[df_merge['PostCode'].isna(),].index.to_list()

## Merge Data Between Raw Data and Postcode Data Without Postcode

In [88]:
df_merge_drop_postcode = df_raw.loc[df_raw['number'].isin(list_not_match)].merge(
    df_code[['PostCode', 'TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai']],
    left_on=['sub_district_name', 'district_name', 'province_name'],
    right_on=['TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai'],
    how='left'
)

## Replace Postcode from Postcode Data to Raw Data

In [89]:
df_merge.loc[df_merge['number'].isin(list_not_match),'postcode'] = df_merge_drop_postcode.loc[df_merge_drop_postcode['number'].isin(list_not_match),'PostCode'].to_list()

# Remove Redundancy Address

## Split Full Address

In [90]:
df_address = df_merge['full_address'].str.split(" ", expand=True)

## List Data Pattern Needed to Remove

In [91]:
list_replace = [
    'ต.',
    'อ.',
    'จ.',
    'เขต',
    'แขวง'
]

## Remove Data That Match With The Pattern in List

In [92]:
for i in range(df_address.shape[0]):
    for j in range(df_address.shape[1]):
        cell = df_address.iloc[i, j]
        if isinstance(cell, str):
            if any(cell.find(word) == 0 for word in list_replace):
                print(cell)
                df_address.iloc[i, j] = None
    df_merge.loc[i, 'full_address'] = " ".join(x for x in df_address.iloc[i] if isinstance(x, str))

ต.บางจาก
อ.พระประแดง
ต.บางจาก
อ.พระประแดง
ต.บางจาก
อ.พระประแดง
ต.บางจาก
อ.พระประแดง
ต.บางจาก
อ.พระประแดง
ต.บางตะเคียน
อ.สองพี่น้อง
แขวงท่าข้าม
เขตบางขุนเทียน
อ.เมือง
ต.ทุ่งนาทัย
อ.ทัพทัน
จ.อุทัยธานี
ต.พาณิชย์
ต.บ้านไผ่
ต.บ้านไผ่
ต.บาวคูเวียง
อ.บางกรวย
จ.นนทบุรี11130
ต.รับร่อ
อ.ท่าแซะ
ต.แหลมงอบ
ต.นาพญา
ต.ท่าตูม
อ.ศรีมหาโพธิ
จ.ปราจีนบุรี
ต.ท่าตูม
อ.ศรีมหาโพธิ
จ.ปราจีนบุรี
ต.ท่าตูม
อ.ศรีมหาโพธิ
จ.ปราจีนบุรี
เขตบางกะปิ
ต.บ้านชบ
อ.สังขะ
จ.สุรินทร์
ต.คลองจินดา
อ.สามพราน
จ.นครปฐม
ต.คลองจินดา
อ.สามพราน
จ.นครปฐม
ต.คลองจินดา
อ.สามพราน
จ.นครปฐม
ต.ทุ่งเบญจา
อ.ท่าใหม่
จ.จันทบุรี


In [94]:
df_merge[0:-5].to_excel('/workspaces/cleandata/export_files/20250701_cleaned_data.xlsx')